In [1]:
import pandas as pd
import pymc3 as pm
import os
import numpy as np
os.getcwd()

'C:\\Users\\lnajt\\Documents\\GitHub\\ErdosInstitute\\ErdosInstitute-SIG_Project\\Feature_Design\\sandbox'

In [2]:
subreddit = "WallStreetBets"
df = pd.read_csv("../../Data/cleaned_" + subreddit + ".csv")

In [3]:
df.groupby("author").agg("count").sort_values(by = 'id')

,Unnamed: 0,id,score,ups,downs,gilded,upvote_ratio,total_awards_received,num_comments,weektime,...,award_Yikes,award_Wearing is Caring,award_Cool S,award_You Don't Say?,award_Take My Money,award_Made Me Smile,award_Table Flip,award_Wait What?,award_Bull,award_Canberry
author,,,,,,,,,,,,,,,,,,,,,
----The_Truth-----,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
cult0cage,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
culdnthinkofanything,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
cuestar14,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
cudi216,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1poundbookingfee,73,73,73,72,72,73,73,73,73,73,...,73,73,73,73,73,73,73,73,73,73
rawrtherapy,77,77,77,74,74,77,77,77,77,77,...,77,77,77,77,77,77,77,77,77,77
bigbear0083,84,84,84,84,84,84,84,84,84,84,...,84,84,84,84,84,84,84,84,84,84


In [4]:
author_list = list(set(df.author))
author_list

['bosshax',
 'bjt23',
 'InTheHamIAm',
 'turog2018',
 'ADAWG9393',
 '84Gothamrogues',
 'jungle_beets',
 'MyNameCannotBeSpoken',
 'BabyFederer',
 'leviticus04',
 'sottsicles',
 'SoarinMarkov',
 'AnimeSnoopy',
 't0doubleD73',
 'thedirtybarnacle',
 'ChefMilky',
 'somePADguyoverhere',
 'Det-Mike-Scarn',
 'Teetje',
 'ALEXLALKB24',
 'Gingermanns',
 'Cgillss',
 'CSGOW1ld',
 'deenomen',
 'Digitalbws',
 'TheOutsideWindow',
 'DS2100',
 'greggenry1555',
 'gman72098',
 'Kyle_SS',
 'melnotgibson',
 'Onyxstraza',
 'mll1895',
 'jaredspivey',
 'Jobber19',
 'aprizm',
 'davidtaughtyou',
 'Suspicious_Seaweed_8',
 'SpaceEnthusiast',
 'Indigo_smoke',
 'Ragesaucey',
 'giannis_34',
 'drich3',
 'phillynov',
 'TheSheikYerbouti',
 'marcandero',
 'kunashni',
 'CumFlakess',
 'moccabee',
 'Firewind11',
 'purplealmondmilk',
 'stvje',
 'pyinvesting',
 'whiskyteats',
 'wallie71',
 'bergerp111',
 'Cog2020',
 'Pug0fCrydee817',
 'SummertimeInParis',
 'Malturk',
 'calikid9one',
 'Pattyg360',
 'toroawayy',
 'seawelldaddy',

In [5]:
num_authors = len(author_list)
author = df[['author', 'ups', 'downs']].groupby('author').agg('sum')
author = author.dropna()

In [ ]:

with pm.Model() as model:
    empirical_mean = df.upvote_ratio.mean()
    empirical_std = df.upvote_ratio.std()
    
    betas = pm.Beta("betas", mu = empirical_mean, sigma = empirical_std, shape = num_authors)
    binomials = pm.Binomial("binomials", p = betas, n = author["ups"] + author["downs"], observed = author["ups"])
    trace = pm.sample(10000, tune = 1000)
    
    # just replacing this with the explicitely likelihood update from the conjugate prior

<ipython-input-6-221ecec76738>:7: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace = pm.sample(10000, tune = 1000)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [betas]


In [ ]:
trace['betas']

In [ ]:
author['beta_mean'] = np.mean(trace['betas'], axis = 0)


In [ ]:
author.sort_values(by = 'beta_mean', ascending = False).head(15)